In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [13]:
#读取数据，处理数据
data = pd.read_csv("D:\桌面\Train.csv")

In [14]:
#把shares大于10000的都删除.占比<5%
rows_to_drop = data[data[' shares'] > 10000].index
df = data.drop(rows_to_drop)

In [15]:
#手动降维
df = df.drop([" n_non_stop_words"," weekday_is_monday"," weekday_is_tuesday"," weekday_is_wednesday"," weekday_is_thursday"," weekday_is_friday"
                ," weekday_is_saturday"," weekday_is_sunday"], axis=1)

In [16]:
# 将训练数据和测试数据分开
X = df.iloc[:, :-1]
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
y = df.iloc[:, -1]
# 卡方检验特征选择和PCA特征提取
skb = SelectKBest(chi2, k=15)
X_new = skb.fit_transform(X_scaled, y)
pca = PCA(n_components=5)
X_pca = pca.fit_transform(X_new)
X_pca = scaler.fit_transform(X_pca)

In [17]:
# 训练模型并进行十折交叉验证
n_splits = 10
kf = KFold(n_splits=n_splits, shuffle=True, random_state=1)
mae_list = []
rmse_list = []
mape_list = []
for train_index, test_index in kf.split(X_pca):
    X_train, X_test = X_pca[train_index], X_pca[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    #把y值分为两类来进行预测
    y_binary = np.where(y_train > 1500,1,0)
    lr_model = LogisticRegression(max_iter=200)
    lr_model.fit(X_train, y_binary)
    #把预测值映射回去
    y_pred = lr_model.predict(X_test)
    print(y_pred.mean())
    y_pred= np.where(y_pred > 0, 100*12, 100*9)
    mae_list.append(mean_absolute_error(y_test, y_pred))
    rmse_list.append(np.sqrt(mean_squared_error(y_test, y_pred)))
    mape_list.append(np.mean(np.abs((y_test - y_pred) / y_test)) * 100)
    print(np.mean(mape_list))

# 显示结果
print('MAE: %.4f' % np.mean(mae_list))
print('RMSE: %.4f' % np.mean(rmse_list))
print('MAPE: %.4f%%' % np.mean(mape_list))

0.29794313369630976
48.38903288203698
0.2813067150635209
48.38475523555398
0.2980332829046899
47.52066728680071
0.29712556732223905
47.572793436145744
0.3059001512859304
48.37094085427762
0.302874432677761
53.03343671302601
0.29652042360060515
52.236844951051225
0.29712556732223905
52.695169732040654
0.30620272314674735
52.118894050138266
0.2826021180030257
51.502883886652604
MAE: 1178.3752
RMSE: 2047.3408
MAPE: 51.5029%


In [12]:
a=[]
a.append(np.mean(mape_list))
a

[51.32856182496913]